In [1]:

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.stats import pearsonr, spearmanr
import h3pandas
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pickle
import re
import sys
from functools import partial
from pygam import s,f,l,LinearGAM
from scipy.signal import find_peaks

import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()


In [3]:
# trophic_niche = str(sys.argv[1])#'all'#str(sys.argv[1]) #'all' #
# A_=int(sys.argv[2])
# B_=int(sys.argv[3])
# C_=int(sys.argv[4])
# D_=int(sys.argv[5])

trophic_niche = 'Trophic_Niche_Invertivore'
# A_=10
# B_=40
# C_=20
# D_=50


## 01.Read birdwave data

In [95]:
# # 01. Read Data
all_birdwave_data = pd.read_csv(f'../../data/D03.smoothed_peaks/all_smoothed_peaks_spring.csv')
all_birdwave_data = all_birdwave_data[all_birdwave_data['niche_or_level']==trophic_niche]
if 'geometry' in all_birdwave_data.columns:
    del all_birdwave_data['geometry']
    

In [78]:
all_birdwave_data


,h3_02,mean_DOY_max,std_DOY_max,mean_DOY_peak,std_DOY_peak,lng,lat,source_model,season,year,niche_or_level,mean_seasonality,cell_index,year_index,mu_ARR,std_ARR
0,82029ffffffffff,106.400000,11.067972,99.840000,22.584434,-129.851595,70.836600,10_40_20_50,spring,2010,all,NaN,63,0,98.515795,25.634161
1,820837fffffffff,136.941176,46.774081,92.357143,34.331946,6.840723,63.444103,10_40_20_50,spring,2010,all,NaN,188,0,91.563741,36.562363
2,820887fffffffff,165.983607,38.409414,67.422222,11.580722,18.265517,60.172836,10_40_20_50,spring,2010,all,NaN,189,0,73.645659,14.810984
3,820897fffffffff,164.285714,48.196522,40.500000,20.506097,19.237456,57.931110,10_40_20_50,spring,2010,all,NaN,191,0,68.983216,22.892866
4,8208a7fffffffff,135.438931,38.525134,95.827434,25.173202,13.222211,60.705183,10_40_20_50,spring,2010,all,NaN,193,0,87.620640,27.825177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12464,82170ffffffffff,92.052980,7.902563,92.158940,7.784680,160.941932,60.731538,10_100_30_80,spring,2020,all,NaN,534,10,93.082076,9.994174
12465,82172ffffffffff,99.718935,6.586178,99.795858,6.568271,159.184658,63.561736,10_100_30_80,spring,2020,all,NaN,538,10,99.321138,8.418004
12466,821747fffffffff,88.222222,7.922830,88.277778,7.789930,171.578765,61.705432,10_100_30_80,spring,2020,all,NaN,540,10,90.849871,10.185244
12467,821757fffffffff,93.788991,10.436824,94.555556,8.341242,166.693304,59.902461,10_100_30_80,spring,2020,all,NaN,541,10,94.244074,10.774184


In [79]:
BW_peak_each_year = all_birdwave_data.copy()


## 02.Read greenwave data

In [80]:
## Read Greenwave data
GW = []

for year in tqdm(range(2010,2021)):
    try:
        dat = pd.read_csv(f'../../data/D04.environmental_data/GreenWave_agg_h3_02_{year}.csv')
        dat['year'] = year
        GW.append(dat)

    except:
        continue
        
GW = pd.concat(GW, axis=0).reset_index(drop=True)

## calculate delta NDVI
def calc_delta_NDVI(df):
    df['delta_NDVI'] = np.gradient(df['mean_NDVI'].values)
    return df

GW = GW.groupby('h3_02').progress_apply(calc_delta_NDVI).reset_index(drop=True)



  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/2381 [00:00<?, ?it/s]

## 03.Read temperature data

In [81]:
## Read Temperature data
temp = []

for year in tqdm(range(2010,2021)):
    try:
        dat = pd.read_csv(f'../../data/D04.environmental_data/h3_02_temperature_{year}.csv')
        dat['year'] = year
        temp.append(dat)

    except:
        continue
      
temp = pd.concat(temp, axis=0).reset_index(drop=True)

def calc_delta_Temp(df):
    df['delta_tmean'] = np.gradient(df['tmean'].values)
    return df

temp = temp.groupby('h3_02').progress_apply(calc_delta_Temp).reset_index(drop=True)


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/2348 [00:00<?, ?it/s]

## 04.Merge env data with birdwave peak

In [82]:
GW = GW.merge(
    BW_peak_each_year[['h3_02','mean_DOY_max', 'std_DOY_max', 'mean_DOY_peak', 'std_DOY_peak',
                   'source_model', 'season', 'year','niche_or_level','mean_seasonality',
                   'mu_ARR','std_ARR']], 
    on=['h3_02','year'], how='left'
)

temp = temp.merge(
    BW_peak_each_year[['h3_02','mean_DOY_max', 'std_DOY_max', 'mean_DOY_peak', 'std_DOY_peak',
                   'source_model', 'season', 'year','niche_or_level','mean_seasonality',
                   'mu_ARR','std_ARR']], 
    on=['h3_02','year'], how='left'
)


In [83]:
GW

,h3_02,DOY,lng,lat,year,mean_NDVI,std_NDVI,delta_NDVI,mean_DOY_max,std_DOY_max,mean_DOY_peak,std_DOY_peak,source_model,season,niche_or_level,mean_seasonality,mu_ARR,std_ARR
0,820007fffffffff,1.0,32.928017,80.175230,2010,-759.380000,73.05296,-4.780030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,820007fffffffff,2.0,32.928017,80.175230,2010,-764.160030,72.71572,-4.859500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,820007fffffffff,3.0,32.928017,80.175230,2010,-769.099000,72.34729,-5.010985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,820007fffffffff,4.0,32.928017,80.175230,2010,-774.182000,71.95348,-5.151500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,820007fffffffff,5.0,32.928017,80.175230,2010,-779.402000,71.53025,-5.282475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9493303,82e0b7fffffffff,362.0,73.350780,-53.060276,2020,27.858337,601.78290,2.911672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9493304,82e0b7fffffffff,363.0,73.350780,-53.060276,2020,30.845001,602.36170,3.060004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9493305,82e0b7fffffffff,364.0,73.350780,-53.060276,2020,33.978344,603.26640,3.209162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9493306,82e0b7fffffffff,365.0,73.350780,-53.060276,2020,37.263325,604.51270,3.359995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
BW_peak_each_year

,h3_02,mean_DOY_max,std_DOY_max,mean_DOY_peak,std_DOY_peak,lng,lat,source_model,season,year,niche_or_level,mean_seasonality,cell_index,year_index,mu_ARR,std_ARR,mu_ARR_int
0,82029ffffffffff,106.400000,11.067972,99.840000,22.584434,-129.851595,70.836600,10_40_20_50,spring,2010,all,NaN,63,0,98.515795,25.634161,99.0
1,820837fffffffff,136.941176,46.774081,92.357143,34.331946,6.840723,63.444103,10_40_20_50,spring,2010,all,NaN,188,0,91.563741,36.562363,92.0
2,820887fffffffff,165.983607,38.409414,67.422222,11.580722,18.265517,60.172836,10_40_20_50,spring,2010,all,NaN,189,0,73.645659,14.810984,74.0
3,820897fffffffff,164.285714,48.196522,40.500000,20.506097,19.237456,57.931110,10_40_20_50,spring,2010,all,NaN,191,0,68.983216,22.892866,69.0
4,8208a7fffffffff,135.438931,38.525134,95.827434,25.173202,13.222211,60.705183,10_40_20_50,spring,2010,all,NaN,193,0,87.620640,27.825177,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12464,82170ffffffffff,92.052980,7.902563,92.158940,7.784680,160.941932,60.731538,10_100_30_80,spring,2020,all,NaN,534,10,93.082076,9.994174,93.0
12465,82172ffffffffff,99.718935,6.586178,99.795858,6.568271,159.184658,63.561736,10_100_30_80,spring,2020,all,NaN,538,10,99.321138,8.418004,99.0
12466,821747fffffffff,88.222222,7.922830,88.277778,7.789930,171.578765,61.705432,10_100_30_80,spring,2020,all,NaN,540,10,90.849871,10.185244,91.0
12467,821757fffffffff,93.788991,10.436824,94.555556,8.341242,166.693304,59.902461,10_100_30_80,spring,2020,all,NaN,541,10,94.244074,10.774184,94.0


## 05.Define functions

In [85]:

def get_expected_env_value_at_arrival(df, var_, DOY_variable = 'mu_ARR_int'):
    try:
        mean_env_value_at_arrival = df[df['DOY']==df[DOY_variable]][var_].mean()
        return mean_env_value_at_arrival
        
    except Exception as e:
        print(e)
        return None
        
        
def get_exp_ARR_based_on_env(df, env_var, expected_env_var, DOY_variable = 'mu_ARR_int'):
    df = df.sort_values(by='DOY')
    target = df[expected_env_var].iloc[0]
    env_val = df[env_var].values
    mean_actual_arrival_date = np.nanmean(df[DOY_variable])
    
    candidates = []
    
    for i in range(len(env_val)-1):
        if (env_val[i] - target) * (env_val[i+1] - target)<0:
            if (i+1>=mean_actual_arrival_date-30) & (i+1<=mean_actual_arrival_date+30): #within 30 days

                if i+1>=183:
                    continue
                elif i+1<=1:
                    continue
                    
                candidates.append(i+1)
                
    if len(candidates)==0:
        return None
    else:
        the_expected_doy = candidates[0]
        return the_expected_doy

    

## 06. Get GreenTrace Index

In [96]:
for var_ in ['mean_NDVI','delta_NDVI']:
    
    res_list = []
    for uncertainty_sample in tqdm(range(100)):
        
        tmp = GW[GW['season']=='spring']
        tmp['mu_ARR_int'] = np.random.normal(loc=tmp['mu_ARR'], scale=tmp['std_ARR'])
        tmp['mu_ARR_int'] = tmp['mu_ARR_int'].astype('int')
        
        expected_env_value = tmp.groupby('h3_02').progress_apply(partial(get_expected_env_value_at_arrival, var_=var_))
        expected_env_value = expected_env_value.reset_index(drop=False).rename(columns={0:f'expected_{var_}'})
        tmp = tmp.merge(expected_env_value[['h3_02',f'expected_{var_}']], how='left', on='h3_02')
        
        res = tmp.groupby(['h3_02','year']).progress_apply(partial(get_exp_ARR_based_on_env,
                                                            env_var=var_, 
                                                            expected_env_var = f'expected_{var_}'))
        res = res.reset_index(drop=False).rename(columns={0:f'expected_trace_by_{var_}'})
        res['season'] = 'spring'

        res_list.append(res)
        
    res_list = pd.concat(res_list, axis=0)
    res = res_list.groupby(['h3_02','year','season']).agg(
        mean_expected_trace=(f'expected_trace_by_{var_}', np.nanmean), 
        std_expected_trace=(f'expected_trace_by_{var_}', np.nanstd)
        ).reset_index(drop=False)
    res = res.rename(columns={f'mean_expected_trace':f'mean_expected_trace_by_{var_}',
                        f'std_expected_trace':f'std_expected_trace_by_{var_}'})
        
    del res_list
    
    all_birdwave_data = all_birdwave_data.merge(
        res, on=['h3_02','year','season'], how='left'
    )

    

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1525 [00:00<?, ?it/s]

  0%|          | 0/12456 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1525 [00:00<?, ?it/s]

  0%|          | 0/12456 [00:00<?, ?it/s]

In [97]:
all_birdwave_data


,h3_02,mean_DOY_max,std_DOY_max,mean_DOY_peak,std_DOY_peak,lng,lat,source_model,season,year,niche_or_level,mean_seasonality,cell_index,year_index,mu_ARR,std_ARR,mean_expected_trace_by_mean_NDVI,std_expected_trace_by_mean_NDVI,mean_expected_trace_by_delta_NDVI,std_expected_trace_by_delta_NDVI
0,82029ffffffffff,106.400000,11.067972,99.840000,22.584434,-129.851595,70.836600,10_40_20_50,spring,2010,all,NaN,63,0,98.515795,25.634161,119.0,NaN,118.0,NaN
1,820837fffffffff,136.941176,46.774081,92.357143,34.331946,6.840723,63.444103,10_40_20_50,spring,2010,all,NaN,188,0,91.563741,36.562363,110.0,NaN,83.0,NaN
2,820887fffffffff,165.983607,38.409414,67.422222,11.580722,18.265517,60.172836,10_40_20_50,spring,2010,all,NaN,189,0,73.645659,14.810984,NaN,NaN,73.0,NaN
3,820897fffffffff,164.285714,48.196522,40.500000,20.506097,19.237456,57.931110,10_40_20_50,spring,2010,all,NaN,191,0,68.983216,22.892866,NaN,NaN,55.0,NaN
4,8208a7fffffffff,135.438931,38.525134,95.827434,25.173202,13.222211,60.705183,10_40_20_50,spring,2010,all,NaN,193,0,87.620640,27.825177,102.0,NaN,79.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12464,82170ffffffffff,92.052980,7.902563,92.158940,7.784680,160.941932,60.731538,10_100_30_80,spring,2020,all,NaN,534,10,93.082076,9.994174,89.0,NaN,117.0,NaN
12465,82172ffffffffff,99.718935,6.586178,99.795858,6.568271,159.184658,63.561736,10_100_30_80,spring,2020,all,NaN,538,10,99.321138,8.418004,116.0,NaN,103.0,NaN
12466,821747fffffffff,88.222222,7.922830,88.277778,7.789930,171.578765,61.705432,10_100_30_80,spring,2020,all,NaN,540,10,90.849871,10.185244,84.0,NaN,115.0,NaN
12467,821757fffffffff,93.788991,10.436824,94.555556,8.341242,166.693304,59.902461,10_100_30_80,spring,2020,all,NaN,541,10,94.244074,10.774184,78.0,NaN,112.0,NaN


## 07. Get ThermoTrace Index

In [99]:
for var_ in ['tmean','tmax','tmin']:
    
    res_list = []
    for uncertainty_sample in tqdm(range(2)):
        
        tmp = temp[temp['season']=='spring']
        tmp['mu_ARR_int'] = np.random.normal(loc=tmp['mu_ARR'], scale=tmp['std_ARR'])
        tmp['mu_ARR_int'] = tmp['mu_ARR_int'].astype('int')
        
        expected_env_value = tmp.groupby('h3_02').progress_apply(partial(get_expected_env_value_at_arrival, var_=var_))
        expected_env_value = expected_env_value.reset_index(drop=False).rename(columns={0:f'expected_{var_}'})
        tmp = tmp.merge(expected_env_value[['h3_02',f'expected_{var_}']], how='left', on='h3_02')
        
        res = tmp.groupby(['h3_02','year']).progress_apply(partial(get_exp_ARR_based_on_env,
                                                            env_var=var_, 
                                                            expected_env_var = f'expected_{var_}'))
        res = res.reset_index(drop=False).rename(columns={0:f'expected_trace_by_{var_}'})
        res['season'] = 'spring'

        res_list.append(res)
        
    res_list = pd.concat(res_list, axis=0)
    res = res_list.groupby(['h3_02','year','season']).agg(
        mean_expected_trace=(f'expected_trace_by_{var_}', np.nanmean), 
        std_expected_trace=(f'expected_trace_by_{var_}', np.nanstd)
        ).reset_index(drop=False)
    res = res.rename(columns={f'mean_expected_trace':f'mean_expected_trace_by_{var_}',
                        f'std_expected_trace':f'std_expected_trace_by_{var_}'})
        
    del res_list
    
    all_birdwave_data = all_birdwave_data.merge(
        res, on=['h3_02','year','season'], how='left'
    )

    

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1514 [00:00<?, ?it/s]

  0%|          | 0/12376 [00:00<?, ?it/s]

  0%|          | 0/1514 [00:00<?, ?it/s]

  0%|          | 0/12376 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1514 [00:00<?, ?it/s]

  0%|          | 0/12376 [00:00<?, ?it/s]

  0%|          | 0/1514 [00:00<?, ?it/s]

  0%|          | 0/12376 [00:00<?, ?it/s]

In [109]:
all_birdwave_data.iloc[:,-12:].corr(method='spearman')


,mu_ARR,std_ARR,mean_expected_trace_by_mean_NDVI,std_expected_trace_by_mean_NDVI,mean_expected_trace_by_delta_NDVI,std_expected_trace_by_delta_NDVI,mean_expected_trace_by_tmean,std_expected_trace_by_tmean,mean_expected_trace_by_tmax,std_expected_trace_by_tmax,mean_expected_trace_by_tmin,std_expected_trace_by_tmin
mu_ARR,1.000000,-0.219228,0.745654,NaN,0.747440,NaN,0.809911,-0.066630,0.819903,-0.057760,0.836367,-0.061603
std_ARR,-0.219228,1.000000,-0.244029,NaN,-0.207256,NaN,-0.213960,0.061332,-0.225789,0.062107,-0.226152,0.064801
mean_expected_trace_by_mean_NDVI,0.745654,-0.244029,1.000000,NaN,0.574183,NaN,0.688755,-0.029104,0.699995,-0.019336,0.679777,-0.041831
std_expected_trace_by_mean_NDVI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_expected_trace_by_delta_NDVI,0.747440,-0.207256,0.574183,NaN,1.000000,NaN,0.631387,-0.033112,0.638390,-0.023214,0.653376,-0.039298
std_expected_trace_by_delta_NDVI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_expected_trace_by_tmean,0.809911,-0.213960,0.688755,NaN,0.631387,NaN,1.000000,0.069746,0.868409,0.029135,0.863983,0.027010
std_expected_trace_by_tmean,-0.066630,0.061332,-0.029104,NaN,-0.033112,NaN,0.069746,1.000000,0.022631,0.142590,0.001563,0.097539
mean_expected_trace_by_tmax,0.819903,-0.225789,0.699995,NaN,0.638390,NaN,0.868409,0.022631,1.000000,0.076320,0.841177,0.000703
std_expected_trace_by_tmax,-0.057760,0.062107,-0.019336,NaN,-0.023214,NaN,0.029135,0.142590,0.076320,1.000000,0.001058,0.065237


In [112]:
all_birdwave_data
print(all_birdwave_data.isnull().sum()/all_birdwave_data.shape[0])
all_birdwave_data.to_csv(f'../../data/D05.greentrace_thermotrace/{trophic_niche}_curve_query.csv', index=False)


h3_02                                0.000000
mean_DOY_max                         0.000000
std_DOY_max                          0.000000
mean_DOY_peak                        0.000000
std_DOY_peak                         0.000000
lng                                  0.000000
lat                                  0.000000
source_model                         0.000000
season                               0.000000
year                                 0.000000
niche_or_level                       0.000000
mean_seasonality                     1.000000
cell_index                           0.000000
year_index                           0.000000
mu_ARR                               0.000000
std_ARR                              0.000000
mean_expected_trace_by_mean_NDVI     0.318309
std_expected_trace_by_mean_NDVI      1.000000
mean_expected_trace_by_delta_NDVI    0.203705
std_expected_trace_by_delta_NDVI     1.000000
mean_expected_trace_by_tmean         0.015559
std_expected_trace_by_tmean       

In [10]:
all_birdwave_data.columns

Index(['h3_02', 'mean_DOY_max', 'std_DOY_max', 'mean_DOY_peak', 'std_DOY_peak',
       'lng', 'lat', 'source_model', 'season', 'year', 'niche_or_level',
       'mean_seasonality', 'cell_index', 'year_index', 'mu_ARR', 'std_ARR',
       'mean_expected_trace_by_mean_NDVI', 'std_expected_trace_by_mean_NDVI',
       'mean_expected_trace_by_delta_NDVI', 'std_expected_trace_by_delta_NDVI',
       'mean_expected_trace_by_tmean', 'std_expected_trace_by_tmean',
       'mean_expected_trace_by_tmax', 'std_expected_trace_by_tmax',
       'mean_expected_trace_by_tmin', 'std_expected_trace_by_tmin'],
      dtype='object')

In [17]:
all_birdwave_data = pd.read_csv(f'../../data/D05.greentrace_thermotrace/{trophic_niche}_curve_query.csv')
res = all_birdwave_data[all_birdwave_data['lat']>0].groupby('h3_02').apply(lambda x:pd.Series({
    'mean_expected_trace_by_mean_NDVI':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_mean_NDVI'])[0],
    'mean_expected_trace_by_delta_NDVI':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_delta_NDVI'])[0],
    'mean_expected_trace_by_tmean':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmean'])[0],
    'mean_expected_trace_by_tmax':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmax'])[0],
    'mean_expected_trace_by_tmin':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmin'])[0]
}))
print(res.mean(axis=0))

res = all_birdwave_data[(all_birdwave_data['lat']>0) & (all_birdwave_data['lng']<-30)].groupby('h3_02').apply(lambda x:pd.Series({
    'mean_expected_trace_by_mean_NDVI':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_mean_NDVI'])[0],
    'mean_expected_trace_by_delta_NDVI':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_delta_NDVI'])[0],
    'mean_expected_trace_by_tmean':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmean'])[0],
    'mean_expected_trace_by_tmax':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmax'])[0],
    'mean_expected_trace_by_tmin':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmin'])[0]
}))
print(res.mean(axis=0))

res = all_birdwave_data[(all_birdwave_data['lat']>0) & (all_birdwave_data['lng']>-30)].groupby('h3_02').apply(lambda x:pd.Series({
    'mean_expected_trace_by_mean_NDVI':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_mean_NDVI'])[0],
    'mean_expected_trace_by_delta_NDVI':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_delta_NDVI'])[0],
    'mean_expected_trace_by_tmean':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmean'])[0],
    'mean_expected_trace_by_tmax':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmax'])[0],
    'mean_expected_trace_by_tmin':spearmanr(x['mu_ARR'],x['mean_expected_trace_by_tmin'])[0]
}))
print(res.mean(axis=0))


mean_expected_trace_by_mean_NDVI     0.382032
mean_expected_trace_by_delta_NDVI    0.476086
mean_expected_trace_by_tmean         0.568951
mean_expected_trace_by_tmax          0.576061
mean_expected_trace_by_tmin          0.610600
dtype: float64
mean_expected_trace_by_mean_NDVI     0.349415
mean_expected_trace_by_delta_NDVI    0.382832
mean_expected_trace_by_tmean         0.460573
mean_expected_trace_by_tmax          0.483059
mean_expected_trace_by_tmin          0.512859
dtype: float64
mean_expected_trace_by_mean_NDVI     0.395926
mean_expected_trace_by_delta_NDVI    0.509843
mean_expected_trace_by_tmean         0.611982
mean_expected_trace_by_tmax          0.613158
mean_expected_trace_by_tmin          0.649624
dtype: float64
